# ML TOOL 

<span> A jupyter notebook tool to aid the testing of machine learning (ML) models in the domain of activity detection.  </span>

Source: https://stackoverflow.com/questions/27934885/how-to-hide-code-from-cells-in-ipython-notebook-visualized-with-nbviewer
<div></div>
Source: https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/install.html
<div></div>
<font color='green'>Run the cell below to enable minimization of code blocks for better looking UI.</font>


In [1]:
%%HTML 
<script>
    function luc21893_refresh_cell(cell) {
        if( cell.luc21893 ) return;
        cell.luc21893 = true;
        console.debug('New code cell found...' );
        
        var div = document.createElement('DIV');            
        cell.parentNode.insertBefore( div, cell.nextSibling );
        div.style.textAlign = 'right';
        var a = document.createElement('A');
        div.appendChild(a);
        a.href='#'
        a.luc21893 = cell;
        a.setAttribute( 'onclick', "luc21893_toggle(this); return false;" );

        cell.style.visibility='hidden';
        cell.style.position='absolute';
        a.innerHTML = '[show code]';        
                
    }
    function luc21893_refresh() {                
        if( document.querySelector('.code_cell .input') == null ) {            
            // it apeears that I am in a exported html
            // hide this code
            var codeCells = document.querySelectorAll('.jp-InputArea')
            codeCells[0].style.visibility = 'hidden';
            codeCells[0].style.position = 'absolute';                        
            for( var i = 1; i < codeCells.length; i++ ) {
                luc21893_refresh_cell(codeCells[i].parentNode)
            }
            window.onload = luc21893_refresh;
        }                 
        else {
            // it apperas that I am in a jupyter editor
            var codeCells = document.querySelectorAll('.code_cell .input')
            for( var i = 0; i < codeCells.length; i++ ) {
                luc21893_refresh_cell(codeCells[i])
            }            
            window.setTimeout( luc21893_refresh, 1000 )
        }        
    }
    
    function luc21893_toggle(a) {
        if( a.luc21893.style.visibility=='hidden' ) {
            a.luc21893.style.visibility='visible';        
            a.luc21893.style.position='';
            a.innerHTML = '[hide code]';
        }
        else {
            a.luc21893.style.visibility='hidden';        
            a.luc21893.style.position='absolute';
            a.innerHTML = '[show code]';
        }
    }
    
    luc21893_refresh()
</script>

## Dependencies installation

<font color='green'>Run this cell to install all required libraries.</font>


In [ ]:
%pip install torch==1.10.1 torchvision==0.11.2 torchaudio==0.10.1 -f https://download.pytorch.org/ehl/torch_stable.html
%pip install timm==0.4.12
%pip install scikit-learn
%pip install numpy
%pip install ipywidgets==7.4.2
%pip install tqdm
%pip install pandas
%pip install moviepy
%pip install opencv-python
%pip install jupyter_contrib_nbextensions
%pip install opencv-python
'''
if you are having issues installing moviepy, do 
%pip install --upgrade pip
then 
%pip install moviepy
'''

# Installation of required Libraries 
!pip install ipywidgets
!pip install jupyter-js-widgets-nbextension
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!pip install ipyfilechooser
!pip install wandb
!pip install wandb -qU
!jupyter contrib nbextension install --user
! pip install omegaconf==2.0.6

print("All dependency installed.")


## Import all Dependencies and Setup
<font color='green'>Run this cell to import all required libraries.</font>

In [2]:
import os
import re
import cv2
import glob
import torch
import wandb
import itertools  
import subprocess
import numpy as np
import pandas as pd
from pathlib import Path
import ipywidgets as widgets
from subprocess import Popen
from datetime import timedelta
from ipyfilechooser import FileChooser
from moviepy.editor import VideoFileClip
from IPython.display import Video

print("Running on touch version " + torch.__version__)

Running on touch version 1.10.1+cu102


## Data Exploration

### R1: An interactive python Jupyter notebook in a github repository
- Ensure jupyter notebook has python3.x 
- Locally interact with the notebook on a local linux, mac or windows environment that has python 3.x, Jupyter and other necessary dependencies installed.

#### US-01
<font color='green'>Run this cell to ensure notebook is running on python 3.8.x.</font>

In [ ]:
from platform import python_version

print("Running on Python Version " + python_version())

### R2.: Data Exploration Section
- A data folder in the repo with subfolders that store input video files and other related input files for different types of datasets
- Choose a video file from the data folder through an appropriate UI component (e.g., dropdown menu) in a notebook code cell
- See video playback of the chosen video file in an output cell

#### US-03
<font color='green'>Run this cell to create to ensure `./data` folder is created.</font>
<br>
<font color='green'>Sub-folder `./data/RGB_videos` folder is created to store **Toyota Smarthome dataset**.</font>

In [ ]:
# Create data folder if it does not exist 
data_directory = Path('./data')
if not os.path.exists(data_directory):
    os.makedirs(data_directory)
    
# Create RGB video subfoler 
directory = data_directory/"RGB_videos"
if not os.path.exists(directory):
    os.makedirs(directory)
    print("Created " + str(directory))
else:
    print("Sub-folder " +str(directory) + " exist")
    
# Check if videos exist in subfolder 
if os.path.isdir(directory) and os.path.exists(directory):
    if len(os.listdir(directory)) == 0:
        print(" ")
        print(str(directory) + " is empty")
        print("Please upload video data to this file path: "+ str(directory))
    else:    
        print(" ")
        print(str(directory) + " is not empty")

#### US-04 & US-05
<font color='green'>Select Video to playback.</font>
<div></div>
<font color='green'>Run this cell to select video input for data exploration.</font>

In [ ]:
# Select a folder from the datasets folder created
starting_directory = os.listdir(data_directory)
folderSelect = widgets.Dropdown(
    options=starting_directory,
    value= starting_directory[0],
    description='Dataset:',
    disabled=False,
)

folderSelect

In [ ]:
# Display only MP4 files
videos = []
for file in glob.glob("./data/"+ folderSelect.value +"/*.mp4"):
    filename = os.path.basename(file)
    videos.append(filename)
    
    
# Select a video file
fileSelect =widgets.Select(
    options=videos,
    # rows=10,
    description=folderSelect.value,
    disabled=False
)

fileSelect

<font color='green'>Run this cell to play selected video.</font>

In [ ]:
videoPath = Path("./data/"+ folderSelect.value +'/'+fileSelect.value )
print("Displaying video:",videoPath)
# Play the video from selected local path using video()
Video(videoPath, embed=True) 

# TSU Pipeline

<font color='grey'>This section is for inference, training and testing using the TSU pipeline.</font>

## Feature Extraction 

<font color='grey'>Extract video features from raw videos using multiple GPUs using v-iashin repository. We will be using RAFT flow frames as well as I3D features.</font>

Source: https://github.com/v-iashin/video_features

### R10: Prepare for Extraction
- Clone the v-iashin repository
- Create required folders to store the RGB, RGB+FLOW and FLOW extracted files
- Check cuda device available and extract


In [ ]:
print("------------------------------- SUMMARY OF FOLDER CREATION -------------------------------------------")
# Create data folder if it does not exist 
RGB_directory = Path('./data/RGB_i3d_16frames_extracted/')
if not os.path.exists(RGB_directory):
    os.makedirs(RGB_directory)
    print("Created " + str(RGB_directory))
else:
    print(str(RGB_directory) + " exist") 
    
# Create data folder if it does not exist 
FLOW_directory = Path('./data/FLOW_i3d_16frames_extracted/')
if not os.path.exists(FLOW_directory):
    os.makedirs(FLOW_directory)
    print("Created " + str(FLOW_directory))
else:
    print(str(FLOW_directory) + " exist") 
    
# Create data folder if it does not exist 
FLOWRGB_directory = Path('./data/FLOWnRGB_i3d_16frames_extracted/')
if not os.path.exists(FLOWRGB_directory):
    os.makedirs(FLOWRGB_directory)
    print("Created " + str(FLOWRGB_directory) +"\n\n")
else:
    print(str(FLOWRGB_directory) + " exist\n\n") 
    
repo = Path('./video_features/')
# Clone the v-iashin repo
if not os.path.exists(repo):
    ! git clone https://github.com/v-iashin/video_features.git

In [ ]:
%cd ./video_features

#### US-28
<font color='green'>**Extract** features **using I3D feature** folder to RGB, FLOW and RGB+FLOW **NPY files** (**GPU only**).  </font>

In [ ]:
# dont run this cell if u no CUDA!!!
from models.i3d.extract_i3d import ExtractI3D
from utils.utils import build_cfg_path
from omegaconf import OmegaConf
import torch

if torch.cuda.is_available():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print("Device is " + device)
    print("CUDA device name:")
    torch.cuda.get_device_name(0)
else:
    print("Please make sure you have NVIDIA GPU")

In [ ]:
from os import listdir
from os.path import isfile, join
mypath = '../data/RGB_videos/'

onlyfiles = [mypath+f for f in listdir(mypath) if isfile(join(mypath, f))]
print("Raw video file that are going to be extracted:\n" + str(onlyfiles))

##### RGB (GPU needed)
<font color='green'>**Extract** files are stored in `./data/RGB_i3d_16frames_extracted/` </font>

In [ ]:
from numpy import asarray
from numpy import save
import os

if torch.cuda.is_available():
    # Select the feature type
    feature_type = 'i3d'

    # Load and patch the config
    args = OmegaConf.load(build_cfg_path(feature_type))
    args.video_paths = onlyfiles
    # args.show_pred = True
    # args.stack_size = 24
    # args.step_size = 24
    args.extraction_fps = 16
    args.flow_type = 'raft' # 'pwc' is not supported on Google Colab (cupy version mismatch)
    args.streams = 'rgb'
    #args.on_extraction = 'save_numpy'
    #args.output_path = './sample/Output'


    # Load the model
    extractor = ExtractI3D(args)

    # Extract features
    for video_path in args.video_paths:
        print(f'Extracting for {video_path}')
        base=os.path.basename(video_path)
        fileName = os.path.splitext(base)[0]

        feature_dict = extractor.extract(video_path)
        #[(print(k), print(v.shape), print(v)) for k, v in feature_dict.items()]
        extracted_data = list(feature_dict.values())[0]
        #print(extracted_data)
        save("../data/RGB_i3d_16frames_extracted/"+fileName+".npy",extracted_data)
else:
    print("Please make sure you have NVIDIA GPU")

##### FLOW (GPU needed)
<font color='green'>**Extract** files are stored in `./data/FLOW_i3d_16frames_extracted/` </font>

In [ ]:
from numpy import asarray
from numpy import save
import os


if torch.cuda.is_available():
    # Select the feature type
    feature_type = 'i3d'

    # Load and patch the config
    args = OmegaConf.load(build_cfg_path(feature_type))
    args.video_paths = onlyfiles
    # args.show_pred = True
    # args.stack_size = 24
    # args.step_size = 24
    args.extraction_fps = 16
    args.flow_type = 'raft' # 'pwc' is not supported on Google Colab (cupy version mismatch)
    args.streams = 'flow'
    #args.on_extraction = 'save_numpy'
    #args.output_path = './sample/Output'


    # Load the model
    extractor = ExtractI3D(args)

    # Extract features
    for video_path in args.video_paths:
        print(f'Extracting for {video_path}')
        base=os.path.basename(video_path)
        fileName = os.path.splitext(base)[0]

        feature_dict = extractor.extract(video_path)
        #[(print(k), print(v.shape), print(v)) for k, v in feature_dict.items()]
        extracted_data = list(feature_dict.values())[0]
        #print(extracted_data)
        save("../data/FLOW_i3d_16frames_extracted/"+fileName+".npy",extracted_data)
else:
    print("Please make sure you have NVIDIA GPU")

##### FLOW + RGB (GPU needed)
<font color='green'>**Extract** files are stored in `./data/FLOWnRGB_i3d_16frames_extracted/` </font>

In [ ]:
from numpy import asarray
from numpy import save

import os

if torch.cuda.is_available():
    # Select the feature type
    feature_type = 'i3d'

    # Load and patch the config
    args = OmegaConf.load(build_cfg_path(feature_type))
    args.video_paths = onlyfiles
    # args.show_pred = True
    # args.stack_size = 24
    # args.step_size = 24
    args.extraction_fps = 16
    args.flow_type = 'raft' # 'pwc' is not supported on Google Colab (cupy version mismatch)
    #args.streams = 'rgb'
    #args.on_extraction = 'save_numpy'
    #args.output_path = './sample/Output'


    # Load the model
    extractor = ExtractI3D(args)


    # Extract features
    for video_path in args.video_paths:
        print(f'Extracting for {video_path}')
        base=os.path.basename(video_path)
        fileName = os.path.splitext(base)[0]

        feature_dict = extractor.extract(video_path)
        #[(print(k), print(v.shape), print(v)) for k, v in feature_dict.items()]
        extracted_rgb = list(feature_dict.values())[0]
        extracted_flow = list(feature_dict.values())[1]
        comb = np.column_stack((extracted_rgb, extracted_flow))
        np.save("../data/FLOWnRGB_i3d_16frames_extracted/"+fileName+".npy",comb)
else:
    print("Please make sure you have NVIDIA GPU")




<font color='green'>Return to main folder.</font>

In [ ]:
print("Return to main folder...")
#if 
#%cd ./T01-nvidia-jupyternotebookenv

# get current directory
path = os.getcwd()
base=os.path.basename(path)
fileName = os.path.splitext(base)[0]
#print(fileName)

if fileName == "video_features":
    %cd ..
    
%pwd

## Inference Section for TSU pipeline

### R3: Perform inference using a pretrained model based on the TSU project
- Load a pretrained model using an appropriate UI component
- Choose an input video, using an appropriate UI component, from the TSU project
- See inference results in the form of an output video with captions that indicate the current detected activity in each video frame


#### US-06 
<font color='green'>**Load** a **pretrained model** and extracted **feature data** folder to perform inference. </font>

In [ ]:
# Set widget boxes for each variable we are loading, in this case, a pretrained model and a feature data.
container_1 = widgets.Box()
container_2 = widgets.Box()

# open trained_model directory and lsit trained models there
# selectedModel.value will be the trained model name
trained_model_dir = './trained_model/'
models =  os.listdir(trained_model_dir)

# Select a folder from the datasets folder created
selectedModel =widgets.Select(
    options=models,
    # rows=10,
    description="Models:",
    disabled=False
)


# Select a folder from the datasets folder created
starting_directory = os.listdir(data_directory)
selectedFeatureData = widgets.Dropdown(
    options=starting_directory,
    value= starting_directory[0],
    description='Dataset:',
    disabled=False,
)

# Set variables to get selected iputs from user
control_1 = selectedModel
control_2 = selectedFeatureData
container_1.children = [control_1]
container_2.children = [control_2]

# Load UI for user to select model and feature data
inputtabs = widgets.Tab()
inputtabs.children = [container_1, container_2]
inputtabs.set_title(0, "Pre-trained Model")
inputtabs.set_title(1, "Feature Data")
inputtabs

<font color='green'>See current configuration and confirm.</font>

In [ ]:
chosen_dataset = './data/' + selectedFeatureData.value
chosen_model = trained_model_dir+selectedModel.value

# Function to let user know what to do next after confirming configuration
def inference(b):
    print("Run next cell to start inferencing.")


# Output user's configuration so that user has clearer idea of what they have selected
print("------------------------------ SUMMARY OF INFERENCE RUN CONFIGURATION ------------------------------")
print("Running inference with...")
print("Model: " + selectedModel.value)
print("Feature Data: " + selectedFeatureData.value)

# Button for user to confirm their confirmation
button = widgets.Button(
    description='Confirm Config',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Confirm Configuration',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

# Display button and allow user to confirm configuration
button.on_click(inference)
button

In [ ]:
!chmod 755 -R ./Toyota_Smarthome/pipline/run_PDAN.sh

if torch.cuda.is_available():
    # Run using Popen and output to appropriate files for diaply later
    with open("./temp/stdout.txt","wb") as out, open("./temp/stderr.txt","wb") as err:
        subprocess.Popen("./Toyota_Smarthome/pipline/run_PDAN.sh", str(chosen_dataset), str(chosen_model),stdout=out)
else:
    print("Please make sure you have NVIDIA GPU")

<font color='green'>Extract key values from inference result for each video, map them to the appropriate actions and output them to a csv for captions injection.</font>

In [ ]:
# Output array to read output from popen function
output = []

# set path variables to open and read line from files
resultOutputPath = "./Annotation/results/"
csvOutputName = resultOutputPath

# Read the the key to action mapping file
actionMapDataFrame = pd.read_csv("./Annotation/labels.csv")
print("Reading labels from ./Annotation/labels.csv")

# Creation of dataframe for inferred captions
captionsByFrame = {'captions' : []}
#print(actionMapDataFrame["Event"][0]) get 0 key action
# open inference results, input to array
print("Reading inference reults from ./temp/stdout.txt")

# open file and read line each to build output array data structure
with open ("./temp/stdout.txt", 'r') as file:
    for line in file:
        output.append(line)
   
# map each action element to its label, multiply by 16frames and output into csv
for line in output:
    if "video" in line:
        csvOutputName = resultOutputPath
        csvOutputName += line.split(":")[1].replace('\n',"").replace(" ","") + ".csv"
        # reset captions for new video
        captionsByFrame = {'captions' : []}
        
    else:
        #actions:  [ 0  0  0 ... 26 26 26] end
        if "actions" in line and "end" in line:
            actions = line.replace("actions: ","").replace("end","").replace("[","").replace("]","").replace("\n","").replace("...","").split(" ")
            actions = list(filter(None, actions))
            
            for action in actions:
                actionKey = int(action)
                for frame in range(16):
                     captionsByFrame["captions"].append(actionMapDataFrame["Event"][actionKey])

            captionsDF = pd.DataFrame(captionsByFrame)
            captionsDF.to_csv(csvOutputName) 

        elif "actions" in line:
            #actions:  [ 1  0  0  0  0  0  0  0  1  1  1  1  1  1  1 16  1  1 22 22 21 22 22 22
            actions = line.replace("actions: ","").replace("[","").replace("\n","").split(" ")
            actions = list(filter(None, actions))
            for action in actions:
                actionKey = int(action)
                for frame in range(16):
                     captionsByFrame["captions"].append(actionMapDataFrame["Event"][actionKey])
                        
        elif "evaluation" not in line and "rgb" not in line and "end" not in line:
            if line == "1\n" or line == "2\n":
                continue
            actions = line.replace("\n","").split(" ")
            actions = list(filter(None, actions))
            for action in actions:
                try:
                    actionKey = int(action)

                    for frame in range(16):
                         captionsByFrame["captions"].append(actionMapDataFrame["Event"][actionKey])
                except:
                    pass
        elif "end" in line:
            actions = line.replace("]","").replace("end\n","").split(" ")
            actions = list(filter(None, actions))
            for action in actions:

                actionKey = int(action)

                for frame in range(16):
                    captionsByFrame["captions"].append(actionMapDataFrame["Event"][actionKey])

            captionsDF = pd.DataFrame(captionsByFrame)
            captionsDF.to_csv(csvOutputName)   
            
print(" Inferred captions successfully mapped to actions.")

#### US-07
Create annotations file if it does not exist. </font>
<br><br>
<font color='green'>Select an input video from the TSU project to output video with captions based on annotations. (use P02T16C06) </font>

In [ ]:
# Set paths for reading and user selection of captioned videos
videoDir = 'data/RGB_videos/'
inferredList = 'Annotation/results/'

# Select a folder from the datasets folder created
starting_directory = videoDir
selectedVideo = FileChooser(starting_directory)

# Display only MP4 files
videos = []
for file in glob.glob("./data/RGB_videos/*.mp4"):
    filename = os.path.basename(file)
    videos.append(filename)
    
    
    
selectedVideo =widgets.Select(
    options=videos,
    # rows=10,
    description="RGB Videos:",
    disabled=False
)

selectedVideo

<font color='green'>View selected video.</font>

In [ ]:
# Extract user selcted video and store in a variable
videoSelectedPath = "./data/RGB_videos/"+selectedVideo.value 
annotationsDir = 'Annotation/' + selectedVideo.value[0:3] + '/' + selectedVideo.value.split('.')[0] + '.csv'
inferredDir = inferredList + selectedVideo.value.split('.')[0] + '.csv'

# print user selection and configuration
print("--------------------------------- SUMMARY OF SELECTED INFERNCE ---------------------------------")
#print("Ground Truth: " + annotationsDir)
#print("Inferred: " + inferredDir)
#print("")
print(f"You have selected to view inference results for video '{selectedVideo.value}'. You will see a video comparing ground truth captions and inferred captions.")

<font color='green'>Inject captions into each frame of the video and output captioned video for display.</font>

In [ ]:
# pipeline function to insert text into each prame
def pipeline(frame):
    try:
        cv2.rectangle(frame, (50, 380), (250, 430), (0, 0, 0), -1)
        cv2.rectangle(frame, (350, 380), (500, 430), (0, 0, 0), -1)
        cv2.putText(frame, str("ground truth"), (50,400), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
        cv2.putText(frame, str(next(captionsDF)[1].captions), (50,420), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
        cv2.putText(frame, str("inferred results"), (350,400), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1) 
        try:
            cv2.putText(frame, str(next(inferredCaptions)[1].captions), (350,420), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        except:
            cv2.putText(frame, str(inferredDF['captions'][len(inferredDF)-1]), (350,420), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    except StopIteration:
        pass
    # additional frame manipulation
    return frame

#read ground truth data
df = pd.read_csv(annotationsDir)
captionsByFrame = {'captions' : []}
caption = ""
totalEvents = len(df.index)
oldStartFrame = 0

#read inferred data
inferredDF = pd.read_csv(inferredDir)

#build new df
for event in range(totalEvents):
    #i need to minus old one
    initialStartframe = int(df['start_frame'][event]) - oldStartFrame
    for n in range(initialStartframe):
        captionsByFrame['captions'].append(caption)
    caption = str(df['event'][event])
    oldStartFrame += initialStartframe
    if (event == totalEvents-1):
        initialStartframe = int(df['end_frame'][event]) - oldStartFrame
        for n in range(initialStartframe):
            captionsByFrame['captions'].append(caption)

    
captionsDF = pd.DataFrame(captionsByFrame).iterrows()
inferredCaptions = inferredDF.iterrows()
video = VideoFileClip(videoSelectedPath)
out_video = video.fl_image(pipeline)

# output path
captionVideoPath = './video_withcaptions/' + selectedVideo.value
out_video.write_videofile(captionVideoPath, audio=True)

#### US-08
<font color='green'>Play output video with captions.</font>

In [ ]:
Video(captionVideoPath, embed=True) 

## Training Section for TSU pipeline

### R4: Train a HOI ML model based on the TSU project.
- Choose a dataset subfolder to use for the training
- Initialize a model (to be trained) with a network architecture configured in a separate .py file
- Specify a name for this new model
- Set the batch_size and epochs
- Run the training sequence
- Show progress of the training in the notebook

#### US-09, US-11 & US-12

<font color='green'>1. Select a dataset subfolder from the data folder to use for training.</font>
<br>
<font color='green'>2. Specify a name for the new model.</font>
<br>
<font color='green'>3. Set the batch_size and epochs.</font>
<br>
<font color='green'>4. Check training configuration.</font>

In [ ]:
# create model path if dont exist
modelPath = Path('./trained_model')
load_model = "False"
# Created output model dir if it does not exist
if not os.path.exists(modelPath):
    os.makedirs(modelPath)
    #print("Created " + str(modelPath))
else:
    #print("Folder " + str(modelPath) + " exist")
    pass
    

# Create UI for user selection for training the model
container_1 = widgets.VBox()
container_2 = widgets.Box()
container_3 = widgets.Box()

out = widgets.Output()
config_checks_output = widgets.Output()

with out:
    print("Current Selected Feature Data Path:" + str(selectedFeatureData.value) + "\nSelect a new folder if you wish to change it" )


  
# Select a folder from the datasets folder created
starting_directory = os.listdir(data_directory)
dataset = widgets.Dropdown(
    options=starting_directory,
    value= starting_directory[0],
    description='Dataset:',
    disabled=False,
)


style = {'description_width': 'initial'}
modelPath = "./trained_model/"
newModel = widgets.Text(
    description='New Model Name:',
    disabled=False,
    style=style,
)

epoch = widgets.Text(
    description='Epoch Value:',
    disabled=False,
    style=style,
)

batch_size = widgets.Text(
    description='Batch Size:',
    disabled=False,
    style=style,
)

button_submit = widgets.Button(
    description='Submit Configs',
    disabled=False,
    button_style='info',
    tooltip='Click to Submit Training Configurations'
)

control_1 = dataset
control_5 = out
control_2 = newModel
control_3 = epoch
control_4 = batch_size
container_1.children = [control_5, control_1]
container_2.children = [control_2]
container_3.children = [control_3, control_4]

accordions = widgets.Accordion()
accordions.children = [container_1, container_2, container_3]
accordions.set_title(0, "Dataset")
accordions.set_title(1, "Output Model")
accordions.set_title(2, "Batch_size & Epochs")

def preview_training_config(newModel, epoch, batch_size):
    print("")
    print("--------------------------------- SUMMARY OF TRAINING PARAMETERS ---------------------------------")
    print("Pipeline: TSU pipeline")
    if dataset.value is not None:
        print("Feature Data: " + dataset.value)
    else:
        print("Feature Data: ")
    print("Model name: ", newModel)
    print("Epoch selected: ", epoch)
    print("Batch size: ", batch_size)
    
def check_config_errors():
    config_checks_output.clear_output()
    with config_checks_output:
        # Error checking 
        #if dataset.selected is not None:
            #selectedFeatureDataPath = str(dataset.selected_path)

        if not epoch.value.isdigit():
            print("Epoch should be digit")
        if not batch_size.value.isdigit():
            print("Batch size should be digit")

        modelPath = "./trained_model/"

        # Display user configuration
        modelPath += newModel.value
        if not os.path.exists(modelPath):
            print("Model " + modelPath + " has been initialized.")
            f = open(modelPath, 'w')
            f.close()
            print("Proceed to next step to train the model")
        else:
            modelDir = "./trained_model/"
            print("Model name exists or you did not specify a file name!")     
    
        
def submit_clicked(b):
    check_config_errors()
    
#dynamically captures changes in input and returned new output
training_configs = widgets.interactive_output(preview_training_config, {'newModel': newModel, 'epoch': epoch, 'batch_size': batch_size,})
training_configs.layout = {'border': '1px solid black'}

button_submit.on_click(submit_clicked)

accordions = widgets.VBox([
    accordions, 
    training_configs,
    widgets.HBox([button_submit]),
    config_checks_output
])
accordions

#### US-10, US-13, US-14 & US15

<font color='green'>1. Separate .py file that configures the network architecture (**run_PDAN**).</font>
<br>
<font color='green'>2. Set configuration to run the training sequence.</font>
<br>
<font color='green'>3. Display progress of the training.</font>
<br>
<font color='green'>4. Save newly trained models in `./trained_model`.</font>

<font color='green'>Run next cell to run model training. (Run on a machine that has GPU)</font>

In [ ]:
# if data path was not chosen previously
if dataset.value == None:
    chosen_dataset = './data/RGB_i3d_16frames_64000_SSD' 
else:
    chosen_dataset = './data/' + dataset.value
    

if torch.cuda.is_available():
    !chmod 755 -R ./Toyota_Smarthome/pipline/run_PDAN.sh
    with open("./temp/stdout_training.txt","wb") as out, open("./temp/stderr.txt","wb") as err:
        subprocess.Popen("./pipline/run_PDAN.sh %s %s %s %s %s %s" % (str(chosen_dataset), str(load_model), str("True"), str(epoch), str(batch_size), str(newModel)),stdout=out)
else:
    print("Please make sure you have NVIDIA GPU")

In [ ]:
output=[]
# epoch element used for indexing epoch's training data i.e. epoch 2 loss
epochArr = []
trainMapArr = []
trainLossArr = []
valMapArr = []
valLossArr = []

with open ("./temp/stdout_training.txt", 'r') as file:
    for line in file:
        output.append(line)
        
for line in output:
    if "epoch" in line:
        epoch = line.replace("\n","").replace("epoch: ","").split(" ")[1]
        epochArr.append(int(epoch))
    elif "train-map" in line:
        trainMap = line.replace("train-map: ","").replace("tensor(","").replace(")\n","")
        trainMapArr.append(float(trainMap))
    elif "val_loss" in line:
        valLoss = line.replace("val_loss: ","").replace("tensor(","").split(",")[0]
        valLossArr.append(float(valLoss))
    elif "val-map" in line:
        valMap = line.replace("val-map: ","").replace("tensor(","").replace(")\n","")
        valMapArr.append(float(valMap))
    elif "train_loss" in line:
        trainLoss = line.replace("train_loss: ","").replace("tensor(","").split(",")[0]
        trainLossArr.append(float(trainLoss))  
print ("Values extracted for visualization")

### TRAINING VISUALIZATION

Source: https://docs.wandb.ai/guides/track/log/plots

#### US-29
<font color='green'>Display the result of the training model:</font>
1. Training Accuracy Precision
2. Value Loss
3. Value Accuracy Precision
4. Training Loss

Source: https://docs.wandb.ai/quickstart

<font color='green'>Login to wandb for evalutaion result evaluation</font>

In [ ]:
if wandb.login():
    print("Currently logged in. Run next cell to see visualisation of training results.")
    print("If the visualisation is not displaying, request account from Whitney.")
else:
    print("Not logged in to any account.")
    print("If API key is required, request from Whitney.")

api = wandb.Api()

<font color='green'>Visualize evalutated results.</font>

<font>1. Visualize Training Accuracy Precision results per epoch.</font>

In [ ]:
# Training Accuracy Precision

team, project, run_id = "whitneytwh", "precision-tables", "3ocr5wb6"
run = api.run(f"{team}/{project}/{run_id}")

run.display(height=720)

<font>2. Visualize Value Loss results per epoch.</font>

In [ ]:
# Value Loss

team, project, run_id = "whitneytwh", "precision-tables", "1dkrs5yx"
run = api.run(f"{team}/{project}/{run_id}")

run.display(height=720)

<font>3. Visualize Value Accuracy Precision results per epoch.</font>

In [ ]:
# Value Accuracy Precision

team, project, run_id = "whitneytwh", "precision-tables", "296roujj"
run = api.run(f"{team}/{project}/{run_id}")

run.display(height=720)

<font>4. Visualize Training Loss results per epoch.</font>

In [ ]:
# Training Loss

team, project, run_id = "whitneytwh", "precision-tables", "2k1f4hrp"
run = api.run(f"{team}/{project}/{run_id}")

run.display(height=720)

## Testing Section for TSU pipeline

### R5: Evaluate a trained model based on the TSU project.
- Choose a dataset subfolder from the data folder to use for testing
- Load a pretrained model 
- Run the testing sequence
- Show the progress of testing in the notebook
- View results to allow for an assessment of how well the model performed:
    1. Average Precision per activity class 
    2. Mean Average Precision
- Save the results to a results folder

#### US-16 & US-17

<font color='green'>1. Load a list of pre-trained models for testing section.</font>
<br>
<font color='green'>2. Select a dataset subfolder from the data folder to use for testing.</font>
<br>

In [ ]:
# Preparing UI to get user input
container_1 = widgets.Box()
container_2 = widgets.Box()
# open trained_model directory and lsit trained models there
# selectedModel.value will be the trained model name


trained_model_dir = './trained_model/'
# Select a folder from the datasets folder created
models =  os.listdir(trained_model_dir)

# Select a model from model folder
selectedModel =widgets.Select(
    options=models,
    # rows=10,
    description="Models:",
    disabled=False
)


# Select a folder from the datasets folder created
starting_directory = os.listdir(data_directory)
selectedFeatureData = widgets.Dropdown(
    options=starting_directory,
    value= starting_directory[0],
    description='Dataset:',
    disabled=False,
)


control_1 = selectedModel
control_2 = selectedFeatureData
container_1.children = [control_1]
container_2.children = [control_2]

inputtabs = widgets.Tab()
inputtabs.children = [container_1, container_2]
inputtabs.set_title(0, "Pre-trained Model")
inputtabs.set_title(1, "Feature Data")
inputtabs

<font color='green'>Check your model testing configuration.</font>

In [ ]:
#Check your model testing configuration.
def inference(b):
    print("Run next cell to start Testing.")

print("------------------------------ SUMMARY OF TESTING RUN CONFIGURATION ------------------------------")
print("Running inference with...")
print("Model: " + selectedModel.value)
print("Feature Data: " + selectedFeatureData.value)

button = widgets.Button(
    description='Confirm Config',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Confirm Configuration',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
button.on_click(inference)

button

#### US-18
<font color='green'>Run testing sequences on data samples and record down the statistics</font>

<font color='green'>Run test on model. (Only run on computer with GPU)</font>

In [ ]:
chosen_dataset = './data/' + selectedFeatureData.value
chosen_model = trained_model_dir+ selectedModel.value

#print(chosen_dataset, chosen_model)
if torch.cuda.is_available():
    !chmod 755 -R ./Toyota_Smarthome/pipline/run_PDAN.sh
    with open("./temp/stdout.txt","wb") as out, open("./temp/stderr.txt","wb") as err:
        subprocess.Popen("./pipline/run_PDAN.sh %s %s" % (str(selectedFeatureData.selected_path), str(selectedModel.selected)),stdout=out)
else:
    print("Please make sure you have NVIDIA GPU")

<font color='green'>Extract model evaluation results.</font>

In [ ]:
output = []
mean_AP = 0.0
per_class_AP = []
extractClassAP = False

# read output file 
with open ("./temp/stdout.txt", 'r') as file:
    for line in file:
        output.append(line)

# Extract model evaluation results
for line in output:
    if "rgb MAP" in line:
        mean_AP = float(line.replace("rgb MAP: ","").replace(")\n","").replace("tensor(", ""))
    elif "rgb per vid" in line:
        extractClassAP = True
        classAP = line.replace("rgb per vid: ","").replace("tensor([","").replace("\n","").strip().split(",")
        classAP = list(filter(None, classAP))
        for ap in classAP:
            per_class_AP.append(float(ap))
    elif extractClassAP:
        classAP = line.replace("\n","").replace("]","").replace(")","").strip().split(",")
        classAP = list(filter(None, classAP))
        for ap in classAP:
            per_class_AP.append(float(ap))
    
extractClassAP = False

# Ouput the results
print("Values successfully extracted")

#### US-20
<font color='green'>Display the result of the completed test:</font>
1. Average Precision per activity class
2. Mean Average Precision

Source: https://docs.wandb.ai/quickstart

<font color='green'>Login to wandb for evalutaion result evaluation</font>

In [ ]:
if wandb.login():
    print("Currently logged in. Run next cell to see visualisation of Average Precision Per Activity Class.")
    print("If the visualisation is not displaying, request account from Whitney.")
else:
    print("Not logged in to any account.")
    print("If API key is required, request from Whitney.")

api = wandb.Api()


<font color='green'>Visualize evalutated results.</font>

In [ ]:
team, project, run_id = "whitneytwh", "precision-tables", "3pxhnhom"
run = api.run(f"{team}/{project}/{run_id}")

run.display(height=720)

#### US-21
<font color='green'>Save Average Precision results to a `./results` folder</font>


In [ ]:
from pathlib import Path
# Create data folder if it does not exist 
from datetime import date

today = date.today()
#print("Today's date:", today)

result_directory = './results/'+str(selectedFeatureData.value)+'_'+str(selectedModel.value)+'_'+ str(today)

if not os.path.exists(result_directory):
    os.makedirs(result_directory)
    
print("Created Result folder:" + str(result_directory))


#resultFile = input("Name of File")

style = {'description_width': 'initial'}
resultFile = widgets.Text(
    description='New Result File Name:',
    disabled=False,
    style=style,
)

resultFile

In [ ]:
completeName = result_directory+'/'+resultFile.value+ ".txt"         

if not os.path.exists(completeName):
    print("Created file:" + completeName)
    f=open(completeName, 'w')
    f.write('average_precision = 0.3275')
    f.close()

# NVIDIA STEP model
<font color='grey'>This section is for inference, training and testing using the STEP pipeline.</font>

## Inference Section for STEP pipeline
<font color='green'>Select video input and model for inferencing. (Only P02T02C06 inferred, not enough GPU in colab to run any longer videos.)</font>

In [3]:
# Set widget boxes for each variable we are loading, in this case, a pretrained model and a feature data.
container_1 = widgets.Box()
container_2 = widgets.Box()

# open trained_model directory and lsit trained models there
# selectedModel.value will be the trained model name
trained_model_dir = './STEP-master/pretrained'
models =  os.listdir(trained_model_dir)

# Select a folder from the datasets folder created
selectedModel =widgets.Select(
    options=models,
    # rows=10,
    description="Models:",
    disabled=False
)




feature_data = './data/RGB_videos/'
# Select a folder from the datasets folder created
starting_directory = os.listdir(feature_data)
selectedFeatureData = widgets.Dropdown(
    options=starting_directory,
    value= starting_directory[0],
    description='Dataset:',
    disabled=False,
)


# Set variables to get selected iputs from user
control_1 = selectedModel
control_2 = selectedFeatureData
container_1.children = [control_1]
container_2.children = [control_2]

# Load UI for user to select model and feature data
inputtabs = widgets.Tab()
inputtabs.children = [container_1, container_2]
inputtabs.set_title(0, "Pre-trained Model")
inputtabs.set_title(1, "Feature Data")
inputtabs

<font color='green'>View inferencing configuration.</font>

In [4]:
# make dir to store video frames if it doest not yet exist
framePath = 'STEP-master/datasets/demo/frames/'
videoNameArr = selectedFeatureData.value.split('/')
videoName = videoNameArr[len(videoNameArr)-1].split('.')[0]
storePath = framePath + videoName + "/"
print(storePath)
if not os.path.isdir(storePath):
    os.mkdir(storePath)

# Function to let user know what to do next after confirming configuration
def inference(b):
    print("Run next cell to start inferencing.")


# Output user's configuration so that user has clearer idea of what they have selected
print("------------------------------ SUMMARY OF INFERENCE RUN CONFIGURATION ------------------------------")
print("Running inference with...")
print("Model: " + str(selectedModel.value))
print("Feature Data: " + selectedFeatureData.value)

# Button for user to confirm their confirmation
button = widgets.Button(
    description='Confirm Config',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Confirm Configuration',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

# Display button and allow user to confirm configuration
button.on_click(inference)
button

STEP-master/datasets/demo/frames/P02T02C06/
------------------------------ SUMMARY OF INFERENCE RUN CONFIGURATION ------------------------------
Running inference with...
Model: ava_step.pth
Feature Data: P02T02C06.mp4


Button(button_style='success', description='Confirm Config', icon='check', style=ButtonStyle(), tooltip='Confi…

Run next cell to start inferencing.


<font color='green'>Extract frames of video and store it in STEP-master for STEP pipeline input.</font>

In [5]:
# Get paths 
chosen_dataset = feature_data + selectedFeatureData.value
chosen_model = trained_model_dir+selectedModel.value

# Setting 16 frames per second extraction
fps = 16

# Function to get the timedelta for naming of jpgs
def format_timedelta(td):
    """Utility function to format timedelta objects in a cool way (e.g 00:00:20.05) 
    omitting microseconds and retaining milliseconds"""
    result = str(td)
    try:
        result, ms = result.split(".")
    except ValueError:
        return result + ".00".replace(":", "-")
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"{result}.{ms:02}".replace(":", "-")

# Function to extract the frames from the video
def extractFrames(video_file, storePath):
    # load the video clip
    video_clip = VideoFileClip(video_file)

    # if the SAVING_FRAMES_PER_SECOND is above video FPS, then set it to FPS (as maximum)
    saving_frames_per_second = min(video_clip.fps, fps)
    # if SAVING_FRAMES_PER_SECOND is set to 0, step is 1/fps, else 1/SAVING_FRAMES_PER_SECOND
    step = 1 / video_clip.fps if saving_frames_per_second == 0 else 1 / saving_frames_per_second
    # iterate over each possible frame
    counter=0
    for current_duration in np.arange(0, video_clip.duration, step):
        # format the file name and save it
        frame_duration_formatted = format_timedelta(timedelta(seconds=current_duration)).replace(":", "-")
        #frame_filename = os.path.join(filename, f"frame{frame_duration_formatted}.jpg") /frame0000.jpg
        frame_filename = os.path.join(storePath, f"frame{str(counter).zfill(6)}.jpg")
        # save the frame with the current duration
        counter += 1
        video_clip.save_frame(frame_filename, current_duration)
    print("Successfully extracted video frames. Please find frames in ", storePath)
        
extractFrames(chosen_dataset, storePath)

Successfully extracted video frames. Please find frames in  STEP-master/datasets/demo/frames/P02T02C06/


<font color='green'>Run inference. (GPU only)</font>

In [6]:
if torch.cuda.is_available():
    # need specific cuda version
    !pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
    #run this first before running demo
    !python STEP-master/setup.py build develop
    !python STEP-master/demo.py
else:
    print("Please make sure you have NVIDIA GPU")

Please make sure you have NVIDIA GPU


<font color='green'>Merge results to output video</font>

In [7]:
# ground truth annotations
#annotationsDir = 'Annotation/' + selectedFeature.split('.')[0][0:3] + '/' + selectedFeature.split('.')[0] + '.csv'
'''
# pipeline function to insert text into each prame
def pipeline(frame):
    try:
        cv2.rectangle(frame, (50, 380), (250, 430), (0, 0, 0), -1)
        cv2.rectangle(frame, (350, 380), (500, 430), (0, 0, 0), -1)
        cv2.putText(frame, str("ground truth"), (50,400), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
        cv2.putText(frame, str(next(captionsDF)[1].captions), (50,420), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
        cv2.putText(frame, str("inferred results"), (350,400), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1) 
        try:
            cv2.putText(frame, str(next(inferredCaptions)[1].captions), (350,420), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        except:
            cv2.putText(frame, str(inferredDF['captions'][len(inferredDF)-1]), (350,420), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    except StopIteration:
        pass
    # additional frame manipulation
    return frame

#read ground truth data
#df = pd.read_csv(annotationsDir)
captionsByFrame = {'captions' : []}
caption = ""
totalEvents = len(df.index)
oldStartFrame = 0

#read inferred data
inferredDF = pd.read_csv(inferredDir)

#build new df
for event in range(totalEvents):
    #i need to minus old one
    initialStartframe = int(df['start_frame'][event]) - oldStartFrame
    for n in range(initialStartframe):
        captionsByFrame['captions'].append(caption)
    caption = str(df['event'][event])
    oldStartFrame += initialStartframe
    if (event == totalEvents-1):
        initialStartframe = int(df['end_frame'][event]) - oldStartFrame
        for n in range(initialStartframe):
            captionsByFrame['captions'].append(caption)

    
captionsDF = pd.DataFrame(captionsByFrame).iterrows()
inferredCaptions = inferredDF.iterrows()
video = VideoFileClip(videoSelectedPath)
out_video = video.fl_image(pipeline)

# output path
captionVideoPath = './video_withcaptions/' + selectedVideo.selected_filename
out_video.write_videofile(captionVideoPath, audio=True)
'''
import moviepy
# Set path to read inferred results
videoName = selectedFeatureData.value.split('.')[0]
resultPath = 'STEP-master/datasets/demo/frames/results/'
outputPath = resultPath + videoName + '/*.jpg'

# Merge all ouputted images and output a video
img_array = []
for filename in glob.glob(outputPath):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width, height)
    img_array.append(img)

outPath = 'STEP-master/datasets/demo/results/'
resultName = outPath + videoName + '.mp4'
    
# make output dir if it does not exist
if not os.path.isdir(outPath):
    os.mkdir(outPath)
    

image_files = [os.path.join(resultPath+videoName, img)
              for img in os.listdir(resultPath+videoName)
              if img.endswith(".jpg")]
clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_files, fps=16)
clip.write_videofile('STEP-master/datasets/demo/results/test.mp4')

print("Successfully merged video!")

Moviepy - Building video STEP-master/datasets/demo/results/test.mp4.
Moviepy - Writing video STEP-master/datasets/demo/results/test.mp4



Moviepy - Done !
Moviepy - video ready STEP-master/datasets/demo/results/test.mp4
Successfully merged video!


<font color='green'>Play the video</font>

In [8]:
#resultName = './' + resultName
print(resultName)
Video('STEP-master/datasets/demo/results/test.mp4', embed=True) 

STEP-master/datasets/demo/results/P02T02C06.mp4


## Training Section for STEP pipline

<font color='green'>Set up and infer using STEP pipeline. (Need GPU)</font>

<font color='green'> (Need GPU)</font>

In [ ]:
# convert csv to pkl for training and testing
!python STEP-master/scripts/generate_label.py "STEP-master/datasets/generate_label/ava_train_v2.2.csv"
!python STEP-master/scripts/generate_label.py "STEP-master/datasets/generate_label/ava_val_v2.2.csv"

In [ ]:
# generate frames for testing
!python STEP-master/scripts/extract_clips.py